# Postgres 嵌入

> [Postgres Embedding](https://github.com/neondatabase/pg_embedding) 是一个开源的 Postgres 向量相似性搜索扩展，它使用 `Hierarchical Navigable Small Worlds (HNSW)` 进行近似最近邻搜索。

>它支持：
>- 使用 HNSW 进行精确和近似最近邻搜索
>- L2 距离

本 Notebook 展示了如何使用 Postgres 向量数据库 (`PGEmbedding`)。

> PGEmbedding 集成会为您创建 pg_embedding 扩展，但您需要运行以下 Postgres 查询来添加它：
```sql
CREATE EXTENSION embedding;
```

In [ ]:
# Pip install necessary package
%pip install --upgrade --quiet  langchain-openai langchain-community
%pip install --upgrade --quiet  psycopg2-binary
%pip install --upgrade --quiet  tiktoken

将 OpenAI API 密钥添加到环境变量中以使用 `OpenAIEmbeddings`。

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:········


In [3]:
## Loading Environment Variables
from typing import List, Tuple

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGEmbedding
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [5]:
if "DATABASE_URL" not in os.environ:
    os.environ["DATABASE_URL"] = getpass.getpass("Database Url:")

Database Url:········


In [6]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
connection_string = os.environ.get("DATABASE_URL")
collection_name = "state_of_the_union"

In [13]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

## 在 Postgres 中使用 Vectorstore

### 在 PG 中上传向量库

In [32]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
    pre_delete_collection=False,
)

### 创建 HNSW 索引
默认情况下，该扩展执行顺序扫描搜索，召回率为 100%。您可以考虑创建一个 HNSW 索引来进行近似最近邻 (ANN) 搜索，以加快 `similarity_search_with_score` 的执行时间。要在向量列上创建 HNSW 索引，请使用 `create_hnsw_index` 函数：

In [ ]:
PGEmbedding.create_hnsw_index(
    max_elements=10000, dims=1536, m=8, ef_construction=16, ef_search=16
)

上面的函数等同于执行以下 SQL 查询：
```sql
CREATE INDEX ON vectors USING hnsw(vec) WITH (maxelements=10000, dims=1536, m=3, efconstruction=16, efsearch=16);
```
上面语句中使用的 HNSW 索引选项包括：

- maxelements：定义索引的最大元素数量。这是一个必需参数。上面示例的值为 3。真实世界的示例值会大得多，例如 1000000。一个“元素”是指数据集中（向量）的一个数据点，在 HNSW 图中表示为一个节点。通常，您会将此选项设置为一个能够容纳您数据集中行数的数值。
- dims：定义向量数据的维度数量。这是一个必需参数。上面示例中使用的值为小数值。如果您存储的是使用 OpenAI 的 text-embedding-ada-002 模型生成的数据，该模型支持 1536 个维度，那么您应定义值为 1536，例如。
- m：定义在图构造过程中为每个节点创建的双向链接（也称为“边”）的最大数量。
还支持以下附加索引选项：

- efConstruction：定义在索引构造过程中考虑的最近邻数量。默认值为 32。
- efsearch：定义在索引搜索过程中考虑的最近邻数量。默认值为 32。
有关如何配置这些选项来影响 HNSW 算法的信息，请参阅 [Tuning the HNSW algorithm](https://neon.tech/docs/extensions/pg_embedding#tuning-the-hnsw-algorithm)。

### 在 PG 中检索向量存储区

In [15]:
store = PGEmbedding(
    connection_string=connection_string,
    embedding_function=embeddings,
    collection_name=collection_name,
)

retriever = store.as_retriever()

In [16]:
retriever

VectorStoreRetriever(vectorstore=<langchain_community.vectorstores.pghnsw.HNSWVectoreStore object at 0x121d3c8b0>, search_type='similarity', search_kwargs={})

In [17]:
db1 = PGEmbedding.from_existing_index(
    embedding=embeddings,
    collection_name=collection_name,
    pre_delete_collection=False,
    connection_string=connection_string,
)

query = "What did the president say about Ketanji Brown Jackson"
docs_with_score: List[Tuple[Document, float]] = db1.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)